In [ ]:
import pandas as pd 

## Загрузка данных

Загружаем данные из двух источников:
1. `charts.csv` - данные Billboard Charts
2. `top_100_songs_1952_to_2024.xlsx` - данные UK Charts

Excel файл содержит несколько листов, каждый представляет определенный временной период.

In [ ]:
bb = pd.read_csv("charts.csv")
uk_excel = pd.read_excel("top_100_songs_1952_to_2024.xlsx", sheet_name=None)
for name, df in uk_excel.items():
    df["sheet"] = name

uk = pd.concat(uk_excel.values(), ignore_index=True)


for name, df in {"Billboard": bb, "UK": uk}.items():
    print(f"\n{name}:")
    display(df.head())
    display(df.info())


## Нормализация данных

Приводим данные к единому формату:
1. Унификация названий столбцов
2. Добавление информации о стране
3. Стандартизация форматов даты
4. Создание уникальных идентификаторов треков

In [ ]:
bb = bb.rename(columns={"song": "title", "artist": "artist",
                        "date": "date", "peak-rank": "peak_rank"})
uk = uk.rename(columns={"Song": "title",
                        "Artist": "artist", "Week": "date", "Peak": "peak_rank", "Weeks on Chart": "weeks_on_chart", "sheet": "date_source", "Position": "rank"})

bb["country"] = "US"
uk["country"] = "UK"


In [ ]:
for name, df in {"Billboard": bb, "UK": uk}.items():
    print(f"\n{name}:")
    display(df.head())
    display(df.info())

In [ ]:
bb["date"] = pd.to_datetime(bb["date"], format="%Y-%m-%d")
uk["date_str"] = uk["date_source"].astype(str).str.extract(r"(\d{8})")[0]
uk["date"] = pd.to_datetime(uk["date_str"], format="%Y%m%d", errors="coerce")
uk.drop(columns=["date_str"], inplace=True)

In [ ]:
print(bb.shape, uk.shape)

In [ ]:
for df in (bb, uk):
    # строки, где одновременно пустое название или исполнитель – пустота
    df.dropna(subset=["title", "artist"], inplace=True)

    # (title, artist, date) часто уникальны: сними дубликаты
    df.drop_duplicates(subset=["title", "artist", "date"], inplace=True)

In [ ]:
print(bb.shape, uk.shape)

In [ ]:
def make_id(s):
    """
    Создает нормализованный идентификатор для строки текста.
    
    Параметры:
    s (pandas.Series): Серия строк для нормализации
    
    Возвращает:
    pandas.Series: Нормализованные строки
    
    Процесс нормализации:
    1. Приведение к нижнему регистру
    2. Удаление диакритических знаков
    3. Удаление специальных символов
    4. Нормализация пробелов
    """
    return (s.str.lower()
              .str.normalize("NFKD")       # убираем диакритику
              .str.replace(r"[^\w\s]", "", regex=True)
              .str.replace(r"\s+", " ", regex=True)
              .str.strip())

# Создаем уникальные идентификаторы треков
for df in (bb, uk):
    df["track_id"] = make_id(df["title"]) + " - " + make_id(df["artist"])

In [ ]:
uk["year"] = uk["date"].dt.year
uk["week"] = uk["date"].dt.isocalendar().week
bb["year"] = bb["date"].dt.year
bb["week"] = bb["date"].dt.isocalendar().week

In [ ]:
bb.to_parquet("billboard_clean.parquet", index=False)
uk.to_parquet("uk_clean.parquet", index=False)

In [ ]:
combined = pd.concat([bb, uk], ignore_index=True)

print("Уникальных треков:", combined["track_id"].nunique())
print(combined.groupby("country")["year"].agg(["min", "max"]))

In [ ]:
top = (combined
       .groupby(["track_id", "title", "artist", "country", "year"])
       .agg(peak_rank=("peak_rank", "min"),
            weeks_on_chart=("peak_rank", "count"))
       .reset_index())

In [ ]:
songs = (combined[["track_id", "title", "artist"]]
         .drop_duplicates()
         .reset_index(drop=True))
songs.to_csv("songs_for_spotify.csv", index=False)